<a href="https://colab.research.google.com/github/dpostolovski/eeg_is/blob/train_compare_full_data/Final_Countdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title Инсталирање и вчитување на потребните библиотеки

# Библиотека за истражување, визуелизација и анализирање на човечки 
# неврофизиолошки податоци (EEG, sEEG и др)
!pip install mne 
!pip install termcolor
!mkdir saved_models

%tensorflow_version 1.12.0

     |████████████████████████████████| 6.6MB 6.4MB/s 
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [0]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
from termcolor import colored
import mne
from sklearn.decomposition import PCA, FastICA
import mne
from sklearn import metrics 
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.model_selection import train_test_split
import scipy.io as sio
from scipy.fft import fft
from scipy import signal
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras import backend as K
from tensorflow import keras
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.optimizers import Adam
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas.util.testing as tm

import sys
sys.path.append('drive/My Drive/Интелигентни Системи')
from EEGModels import DeepConvNet, EEGNet

K.set_image_data_format('channels_first')

In [4]:
df=pd.read_csv('drive/My Drive/Интелигентни Системи/output_format.csv')
#df.iat[0,0] = 5
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
1,1,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
2,1,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
3,2,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
4,2,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
5,2,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
6,3,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
7,3,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
8,3,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
9,4,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN


In [10]:
# Иницијализација на променливите каде ќе бидат вчитани податоците
data = []
labels = []
events = []
targets = []

for i in range(1, 2): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 1 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.3126 - acc: 0.1724
Epoch 00001: val_loss improved from inf to 2.21223, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 2s 573us/sample - loss: 2.3062 - acc: 0.1722 - val_loss: 2.2122 - val_acc: 0.1133
Epoch 2/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.1081 - acc: 0.1953
Epoch 00002: val_loss improved from 2.21223 to 2.19983, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 221us/sample - loss: 2.1051 - acc: 0.1964 - val_loss: 2.1998 - val_acc: 0.1542
Epoch 3/400
3400/3600 [======

KeyboardInterrupt: ignored

In [0]:
def reshape_data_to_mne_format(data):
  mne_array = np.swapaxes(data, 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0], 1, 8, 350)

  return mne_array

In [9]:
# Иницијализација на променливите каде ќе бидат вчитани тест податоците
test_data = []
test_events = []
test_runs_per_block = [[i for i in range(3)] for j in range(15)] # Covek, Sesija

for i in range(1, 2): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session,block+2] = prediction
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (1) ======================
Вчитување тест податоци од испитниот примерок 1...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 1 се вчитани.

SBJ00| Test_data: (8, 350, 8000)
SBJ00| Test_events: 8000
SBJ00 / S00| Runs per block: 6
SBJ00 / S01| Runs per block: 7
SBJ00 / S02| Runs per block: 7
SBJ00| Predictions: 8000
============== Сесија (0) ==============
5,5,2,5,2,2,2,2,5,5,5,4,6,4,4,5,1,2,2,6,6,5,2,2,6,4,2,5,7,5,2,5,5,5,2,2,2,2,4,2,2,7,5,5,2,2,4,4,4,0,
============== Сесија (1) ==============
2,2,1,0,1,0,5,5,1,1,5,1,2,2,1,2,0,2,2,0,1,2,1,1,2,2,5,2,0,2,1,2,1,5,1,1,1,4,5,1,2,2,2,1,1,1,2,2,5,1,
============== Сесија (2) ==============
2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,5,2,2,2,2,2,5,2,2,5,2,5,2,2,2,2,5,2,2,2,5,2,2,2,6,2,2,2,2,2,2,5,
Stigna li do kraj: True
====================== Примерок (1) ======================




In [0]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,5,5,2,2,2,2,0,2,5,5,5,3,6,3,3,5,3,2,2,6,6,7,7,7,6,3,7,3,7,5,6,5,5,5,2,6,6,6,3,2,2,7,5,3,7,7,3,4,4,6,NaN
1,1,2,2,2,2,0,7,0,2,5,1,1,2,1,6,2,1,2,0,0,2,0,2,2,0,0,2,0,2,2,0,0,0,2,0,2,7,3,1,4,5,1,2,2,2,0,7,3,2,2,5,7,NaN
2,1,3,2,2,6,6,6,2,2,2,6,6,6,2,2,6,2,2,2,2,2,2,2,2,6,2,2,2,2,2,2,7,2,2,2,2,2,2,2,2,2,6,2,2,6,2,2,0,2,6,0,2,NaN
3,2,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
4,2,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
5,2,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
6,3,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
7,3,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
8,3,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
9,4,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
